In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as const
import scipy.fftpack as ft
from numpy import linalg as LA
import time
from matplotlib import animation
from IPython.display import HTML

In [2]:
class state:
    def __init__(self,initial,V,Ekin):
        self.ini=initial
        self.Psi=initial
        self.V=V
        self.Ekin=Ekin
    def advance(self,dt):
        Psi = np.exp(-1j*self.V*dt)*self.Psi        # advance in real-space
        Psik = ft.fftshift(ft.fft(self.Psi))           # Fourier transform
        Psik = np.exp(-1j*self.Ekin*dt)*Psik      # advance in Fourier space
        self.Psi = ft.ifft(ft.fftshift(Psik))          # return to real space
    def get(self):
        return self.Psi
    def reset(self):
        self.Psi=self.ini

In [3]:
hbar = 1
m = 938.28/0.511/2

L = 30
Ngrid = 600
#x space
dx = L/(Ngrid)
x = np.linspace(dx,L,Ngrid)

#k space
k = ft.fftfreq(x.size,d = dx)
k = 2*np.pi*ft.fftshift(k)
dk = np.diff(k).mean()
k0=0
# define potential (in real space) and kinetic energy (in k-space)

## potential well with high walls
_,Vpot=np.loadtxt('data/H2p_pot_gerade.dat',unpack=True)

Ekin = hbar**2*k**2/(2*m)

_,Psi = np.loadtxt('data/H2nuclwf.dat',unpack=True)
Psi = Psi/np.sqrt(np.sum(np.abs(Psi)**2))
Psik = ft.fftshift(ft.fft(Psi))
Psik = np.roll(Psik,np.int(np.round(k0/dk)))
Psi = ft.ifft(ft.fftshift(Psik))

Psi=state(Psi[:Ngrid],Vpot[:Ngrid],Ekin)
dt=5e-1

In [ ]:
fig, ax = plt.subplots()
# add axis specifications
line, = ax.plot([],[],label='real part')

plt.ylim(-.1,.5)
plt.xlim(0,4.5)
plt.legend()
plt.xlabel('x')
#plt.ylabel('wave function')
plt.title('Wave packet propagation without IR laser')
Psi.reset()
def animate(t):
    # assign current wave function to y
    # x is gloabaly defined
    Psi.advance(dt)
    line.set_data(x,np.real(Psi.get()))
    return (line,)

anim = animation.FuncAnimation(fig, animate,
                               frames=np.arange(0,200,dt), # t-values
                               interval=50, # wait time before displaying new frame in ms
                               blit=True)
anim.save('propagation.mp4')
HTML(anim.to_jshtml())